# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,15.29,63,75,2.57,CA,1691387596
1,1,newman,37.3138,-121.0208,28.99,59,0,1.79,US,1691387386
2,2,ushuaia,-54.8000,-68.3000,-3.96,94,99,2.30,AR,1691386970
3,3,mizdah,31.4451,12.9801,22.35,44,0,1.17,LY,1691387447
4,4,edinburgh of the seven seas,-37.0676,-12.3116,11.69,57,28,10.92,SH,1691387188
...,...,...,...,...,...,...,...,...,...,...
581,581,edgewater,28.9889,-80.9023,28.13,93,76,1.92,US,1691387962
582,582,nassau,40.7507,-73.5829,24.82,84,86,1.79,US,1691387963
583,583,greymouth,-42.4667,171.2000,7.56,79,60,3.83,NZ,1691387964
584,584,ixtapa,20.7000,-105.2000,26.24,84,100,0.45,MX,1691387964


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    scale = 0.1
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] > 10) &
                               (city_data_df['Max Temp'] < 30) &
                               (city_data_df['Humidity'] < 30) &
                               (city_data_df['Cloudiness'] <= 40)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,novyy urgal,51.0736,132.5642,27.16,29,17,1.28,RU,1691387657
267,267,namtsy,62.7161,129.6658,24.93,28,33,3.38,RU,1691387759
324,324,charef,34.6210,2.7950,21.96,29,0,3.11,DZ,1691387794
358,358,san juan,-31.5375,-68.5364,13.12,27,7,5.08,AR,1691387818
361,361,g'uzor,38.6214,66.2522,29.02,19,0,4.55,UZ,1691387820
448,448,alice springs,-23.7000,133.8833,21.75,24,0,2.57,AU,1691387880
531,531,biloela,-24.4167,150.5000,23.87,29,2,5.91,AU,1691387932


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ['City', 'Country','Lat', 'Lng', 'Humidity' ]
hotel_df = city_data_df[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
102,novyy urgal,RU,51.0736,132.5642,29,
267,namtsy,RU,62.7161,129.6658,28,
324,charef,DZ,34.6210,2.7950,29,
358,san juan,AR,-31.5375,-68.5364,27,
361,g'uzor,UZ,38.6214,66.2522,19,
448,alice springs,AU,-23.7000,133.8833,24,
531,biloela,AU,-24.4167,150.5000,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    'categories': categories,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
novyy urgal - nearest hotel: No hotel found
namtsy - nearest hotel: No hotel found
charef - nearest hotel: No hotel found
san juan - nearest hotel: Hotel San Francisco
g'uzor - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
biloela - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
102,novyy urgal,RU,51.0736,132.5642,29,No hotel found
267,namtsy,RU,62.7161,129.6658,28,No hotel found
324,charef,DZ,34.6210,2.7950,29,No hotel found
358,san juan,AR,-31.5375,-68.5364,27,Hotel San Francisco
361,g'uzor,UZ,38.6214,66.2522,19,No hotel found
448,alice springs,AU,-23.7000,133.8833,24,Aurora Alice Springs
531,biloela,AU,-24.4167,150.5000,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    tiles = 'OSM',
    geo = True,
    hover_cols=['City', 'Country','Hotel Name'],
    color = 'City',
    size = 'Humidity',
    width = 800,
    height = 600
)

# # Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)